**1**. (25 points)

- Download the data from the URL given into a `pandas` DataFrame (5 points)
```
https://gist.github.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv
```
- Your objective is to predict the `mpg` of a car from `hp`, `wt` and `am`
    - Use the last 10 rows as test data and the rest as training data (5 points)
    - Train a multiple linear regression model on the training data (10 points)
    - Evaluate the mean squared error on the test data (5 points)
    
You may **only** use the following class from `sklearn`  (default parameters are fine)

- `sklearn.linear_model.LinearRegression`

In particular, splitting into test and train data and calculaiton of mean squared error should not use `sklearn`

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
df = pd.read_csv("https://gist.github.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv")

X = df.drop("mpg",axis=1)
y = df['mpg']

X_train = X[:-10]
X_test = X[-10:]
y_train = y[:-10]
y_test = y[-10:]

**2**. (25 points)

- Using the `requests` package, download berries 1, 2, and 3 from `https://pokeapi.co/api/v2/berry` in JSON format and 
convert to a `pandas` DataFrame (5 points)
- Create a new DataFrame that only retains only the `name` column and numeric columns. You should find the appropriate columns, not hard code their locations. (10 points)
- Show only rows where the name begins with the letter `c` (5 points)
- Convert to a `numpy` array (excluding `name`) and standardize so each **row** has mean 0 and standard deviation 1 (5 points)

In [2]:
import requests
import json

berries = [requests.get(f"https://pokeapi.co/api/v2/berry/{x}").json() for x in range(1,4)]
berries

pd.json_normalize(berries, ["flavors"])

,potency,flavor.name,flavor.url
0,10,spicy,https://pokeapi.co/api/v2/berry-flavor/1/
1,0,dry,https://pokeapi.co/api/v2/berry-flavor/2/
2,0,sweet,https://pokeapi.co/api/v2/berry-flavor/3/
3,0,bitter,https://pokeapi.co/api/v2/berry-flavor/4/
4,0,sour,https://pokeapi.co/api/v2/berry-flavor/5/
5,0,spicy,https://pokeapi.co/api/v2/berry-flavor/1/
6,10,dry,https://pokeapi.co/api/v2/berry-flavor/2/
7,0,sweet,https://pokeapi.co/api/v2/berry-flavor/3/
8,0,bitter,https://pokeapi.co/api/v2/berry-flavor/4/
9,0,sour,https://pokeapi.co/api/v2/berry-flavor/5/


In [3]:
df = pd.json_normalize(berries)
numeric = df.select_dtypes("number")
numeric = numeric.join(df["name"]).set_index("name")
numeric

,growth_time,id,max_harvest,natural_gift_power,size,smoothness,soil_dryness
name,,,,,,,
cheri,3,1,5,60,20,25,15
chesto,3,2,5,60,80,25,15
pecha,3,3,5,60,40,25,15


In [4]:
c_numeric = numeric.filter(regex="^c",axis=0)

In [17]:
array = c_numeric.values
array = array.astype(dtype='float')

In [6]:
import numpy as np
row_means = np.mean(c_numeric.values,axis=1)
row_std = np.std(c_numeric.values, axis=1)

In [18]:
for row in range(len(row_means)):
    array[row] -= row_means[row]
    array[row] /= row_std[row]
array

array([[-0.81514658, -0.92081372, -0.70947943,  2.19636716,  0.08302419,
         0.34719206, -0.18114368],
       [-0.84377741, -0.87872677, -0.77387869,  1.14833612,  1.84732332,
        -0.07489149, -0.42438509]])

**3**. (25 points)

We have provided an SQLite3 database in `data/pets.db` with 3 tables `dog`, `treat` and `dog_treat`. The `dog_treat` table is a linker table showing which dog ate which treat. 

- Show a table of ALL dogs and the treats with calories that they ate with column names `dog`, `treat`, `calorie`. A dog that did not eat any treats should still be present in the table (15 points)
- Using a common table expression, show a table with two columns `dog` and `total_calories` where only dogs that have eaten more than 500 calories are displayed (5 points)

In [19]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///data/pets.db")

In [20]:
query = """
SELECT * FROM dog
"""
pd.read_sql(query,con=engine)

,index,dog_id,name,age
0,0,1,Fido,2
1,1,2,Roger,4
2,2,4,Woof,11
3,3,5,Lassie,3


In [21]:
query = """
SELECT * FROM treat
LIMIT5
"""
pd.read_sql(query,con=engine)

,index,treat_id,name,calories
0,0,1,Chewies,100
1,1,2,Bones,500
2,2,3,Yum,1100
3,3,4,Raw Meat,450


In [23]:
query = """
SELECT * FROM dog_treat
LIMIT 5
"""
pd.read_sql(query,con=engine)

,index,treat_id,dog_id
0,0,1,1
1,1,2,1
2,2,2,2
3,3,1,3
4,4,4,3


In [38]:
query = """
SELECT dog.name, treat.name as treat, treat.calories
FROM dog
LEFT JOIN dog_treat
ON dog.dog_id = dog_treat.dog_id
LEFT JOIN treat
ON treat.treat_id = dog_treat.treat_id
"""

pd.read_sql(query,con=engine)

,name,treat,calories
0,Fido,Chewies,100.0
1,Fido,Bones,500.0
2,Roger,Bones,500.0
3,Woof,Bones,500.0
4,Woof,Raw Meat,450.0
5,Lassie,None,NaN


In [46]:
query = """
SELECT dog.name, SUM(treat.calories) as total_calories
FROM dog
LEFT JOIN dog_treat
ON dog.dog_id = dog_treat.dog_id
LEFT JOIN treat
ON treat.treat_id = dog_treat.treat_id
GROUP BY dog.name
HAVING total_calories > 500
"""

pd.read_sql(query,con=engine)

,name,total_calories
0,Fido,600
1,Woof,950


**4**. (40 points)

You want to evaluate whether a liner, quadratic or cubic polynomial is the best model for a set of data using leave-one-out cross-validation (LOOCV) and the mean squared error as evaluation metric. 

- Write a function named `loocv` that takes the predictor variable `x`, the outcome variable `y`, a list of of degrees of polynomial models to be evaluated, and an evaluation function and returns the best model found by LOOCV. For example, you would call the function like this `loocv(x, y, [1,2,3], mse)` where `mse` is of course a function that returns the mean squared error. (30 points)
- Write the `mse` function to provide to the LOOCV routine (5 points)
- Use the `llocv` function to find the best polynomial model for the data provided (5 points)

Notes

- Use the `x` and `y` variables provided
- Do not use any packages except for the standard library and `numpy`
- Code snippets for fitting and evaluation of polynomials is provided

In [47]:
import numpy as np

In [48]:
x = np.load('data/x.npy')
y = np.load('data/y.npy')

In [49]:
coeffs = np.polyfit(x, y, 2)
ypred = np.polyval(coeffs, x)

Object `np.LOOCV` not found.
